In [1]:
import pandas as pd
import os
from tqdm import tqdm
import networkx as nx

In [2]:
tqdm.pandas()

In [3]:
USERS_DIR = '../data/23_17_38_2/users/'

In [4]:
len(os.listdir(USERS_DIR))

1601

### Read all users data

In [5]:
users = os.listdir(USERS_DIR)
data = pd.read_csv(USERS_DIR+users.pop(), sep='\t')

for user in tqdm(users):
    data = data.append(
        pd.read_csv(USERS_DIR+user, sep='\t')
    )

100%|██████████| 1600/1600 [09:39<00:00,  2.76it/s]


In [8]:
def get_retweeted_user(row):
    retweeted_user = None
    if row['retweet']:
        try:
            retweeted_user = row['tweet'].split(' ')[1][1:-1]
        except Exception as e:
            print()
            print(e)
            print(row['tweet'])

    return retweeted_user 

data['retweeted_user'] = data.progress_apply(get_retweeted_user, axis=1)

 52%|█████▏    | 205158/396862 [00:05<00:04, 39890.35it/s]
list index out of range
True

'float' object has no attribute 'split'
nan

'float' object has no attribute 'split'
nan
100%|██████████| 396862/396862 [00:10<00:00, 36125.50it/s]


In [11]:
data.to_csv('../data/23_17_38_2/all_users.csv')

In [12]:
# data = pd.read_csv('../data/23_17_38_2/all_users.csv')

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396862 entries, 0 to 55
Data columns (total 39 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       396862 non-null  object 
 1   id               396861 non-null  object 
 2   conversation_id  396861 non-null  object 
 3   created_at       396861 non-null  object 
 4   date             396860 non-null  object 
 5   time             396860 non-null  object 
 6   timezone         396860 non-null  float64
 7   user_id          396861 non-null  float64
 8   username         396860 non-null  object 
 9   name             396860 non-null  object 
 10  place            18 non-null      object 
 11  tweet            396860 non-null  object 
 12  language         396859 non-null  object 
 13  mentions         396859 non-null  object 
 14  urls             396858 non-null  object 
 15  photos           396858 non-null  object 
 16  replies_count    396858 non-null  float64


### Create graph

In [10]:
users = data['username'].unique()

In [11]:
G = nx.DiGraph()

for user in tqdm(users):
    user_data = data[data['username'] == user]
    rt_users = user_data[user_data['retweet'] == True]['retweeted_user'].unique()
    
    for rt_user in rt_users:
        if rt_user is not None and user is not None:
            G.add_edge(rt_user, user)

100%|██████████| 1602/1602 [00:33<00:00, 47.56it/s]


In [12]:
len(G.nodes)

26371

In [13]:
len(G.edges)

70422

### Take giant component

In [14]:
giant = max(nx.weakly_connected_components(G), key=len)
giant = G.subgraph(giant)

In [15]:
len(giant.nodes)

26335

In [16]:
len(giant.edges)

70403

### Save graph

In [18]:
nx.write_edgelist(giant, '../data/23_17_38_2/full_graph.txt', delimiter=',')